In [12]:
import numpy as np

class Neuron:
    def __init__(self):
        self.type = 'relu'
        self.fp = 0
        self.z = 0
        self.W = []
        self.b = 0
        self.ac_1 = []
        self.dz=0
    
    def _weighted_sum(self):
        summ = 0
        for i in range(len(self.ac_1)):
            summ+=self.ac_1[i]*self.W[i]
        return summ + self.b
    
    def _activation_fn(self):
        if(self.type=='relu'):
            if self.z > 0:
                return self.z
            return 0
        elif(self.type=='sigm'):
            return 1/(1+np.exp(-self.z))
        else:
            raise Exception('Invalid Activation Function')
                    
    def forward_propogate(self,inp,weights,bias,choice='relu'):
        assert(len(inp)==len(weights))
        self.type=choice
        self.ac_1 = inp
        self.W = weights
        self.b = bias
        self.z = self._weighted_sum()
        self.fp = self._activation_fn()
        return self.fp

    def _back_activation_fn(self,dac):
        if(self.type=='relu'):
            if(self.z<0):
                return 0
            else:
                return dac
        elif(self.type=='sigm'):
            return self.dac*self.fp*(1-self.fp)
        else:
            raise Exception("Invalid activation function")

    def back_propogate(self,dac):
        self.dac = dac
        self.dz = self._back_activation_fn(dac)
        m = len(self.ac_1)
        dw = np.array(self.ac_1)*self.dz
        db = self.dz
        dac_1 = np.array(self.W)*self.dz
        return dw,db,dac_1

class Layer:
    def __init__(self, n_neurons):
        self.neurons = []
        self.n_neurons = n_neurons
        for i in range(self.n_neurons):
            self.neurons.append(Neuron())
    
    def forward_propogate(self, inputs, weights,bias,choice='relu'):
        output = []
        for i in range(self.n_neurons):
            output.append(self.neurons[i].forward_propogate(inputs,weights[i],bias[i],choice))
        return output
    
    def back_propogate(self,dacs):
        dacs=np.array(dacs.mean(axis=0))
        dws = []
        dbs = []
        dac_1s = []
        for i in range(dacs.shape[0]):
            temp1,temp2,temp3 = self.neurons[i].back_propogate(dacs[i])
            dws.append(temp1)
            dbs.append(temp2)
            dac_1s.append(temp3)
        return np.array(dws),np.array(dbs),np.array(dac_1s)
    

class Neural_Net:
    def __init__(self, layers):
        self.n_layers=len(layers)
        self.inputs=None
        self.outputs=None
        self.Layers=[]
        self.Weights=[]
        self.WGrads = []
        self.Bias=[]
        self.BGrads = []
        self.layers=layers
        self.learning_rate = 0
        self.n_inps = 0
    
    def _initialize(self):
        for i in range(self.n_layers):
            self.Layers.append(Layer(self.layers[i]))
            if i==0:
                self.Weights.append(np.random.randn(self.layers[i],len(self.inputs[0])))#####
                self.Bias.append(np.random.randn(self.layers[0]))
            else:
                self.Weights.append(np.random.randn(self.layers[i],self.layers[i-1]))
                self.Bias.append(np.random.randn(self.layers[i]))
        self.Layers.append(Layer(1))
        self.Weights.append(np.random.randn(1,self.layers[self.n_layers-1]))
        self.Bias.append(np.random.randn(1))
        
        
    def _showWandB(self):
        print('Weights:',self.Weights)
        print('Bias:',self.Bias)
        
    def _showGWandBG(self):
        print('WGrads:',self.WGrads)
        print('BGrads:',self.BGrads)
        
    def train(self,inps,outs,epochs=50,learning_rate = 0.03,printCost=False):
        self.inputs = inps
        self.outputs = outs
        self.learning_rate = learning_rate
        self._initialize()
        cost_av = 0
        cost_der_av = 0
        self.n_inps = len(self.inputs)
        WGtemp = []
        BGtemp = []
        print(self.predict([4,-2]))
        for i in range(epochs):
            if(printCost and i%50==0):
                print("Epoch %d/%d"%(i+1,epochs),end=" ")
#             self._showWandB()
            for j in range(len(self.inputs)):
                pred = self._forward_propogate(self.inputs[j])
                cost,cost_der = self._compute_cost(pred,self.outputs[j])
                WGtemp,BGtemp=self._back_propogate(cost_der)
                cost_av+=cost
                cost_der_av+=cost_der
                if(j==0):
                    self.WGrads = WGtemp
                    self.BGrads = BGtemp
                else:
                    for k in range(len(self.Layers)):
                        self.WGrads[k]=self.WGrads[k]+WGtemp[k]
                        self.BGrads[k]=self.BGrads[k]+BGtemp[k]
            for j in range(len(self.Layers)):
                self.WGrads[j]=self.WGrads[j]/self.n_inps
                self.BGrads[j]=self.BGrads[j]/self.n_inps
            cost_av/=self.n_inps
            cost_der_av/=self.n_inps
            if(printCost and i%50==0):
                print("Cost = %f, Cost_Der = %f"%(cost_av,cost_der_av))
#             self._showGWandBG()
            self._update_weights()
#         self._showWandB()
#         self._showGWandBG()
                
    def _forward_propogate(self,temp):
        for i in range(len(self.Layers)):
            if i==0:
                temp = self.Layers[i].forward_propogate(temp,self.Weights[i],self.Bias[i],'relu')
            elif i==self.n_layers:
                temp = self.Layers[i].forward_propogate(temp,self.Weights[i],self.Bias[i],choice='sigm')
            else:
                temp = self.Layers[i].forward_propogate(temp,self.Weights[i],self.Bias[i],'relu')
        return temp
    
    def _compute_cost(self,pred,act,cost_type = 'crs_ent'):
        temp_out = np.array(pred)
        act=np.array(act)
        act = np.expand_dims(act,axis=0)
        m = act.shape[0]
        if(cost_type == 'mse'):
            cost = (1/(2*m))*(np.array(temp_out)-np.array(act))**2
            cost_der = np.array(temp_out)-np.array(act)
        elif(cost_type == 'crs_ent'):
            cost = (-1 / m) * (np.multiply(act, np.log(temp_out)) + np.multiply(1 - act, np.log(1 - temp_out)))
            cost_der =  (-1/m)* (np.divide(act, temp_out) - np.divide(1 - act, 1 - temp_out))
        else:
            raise Exception('Invalid cost function given.')
        return cost,cost_der
    
    def _back_propogate(self,cost_der):
        temp3 = np.array(cost_der)
        temp3 = np.expand_dims(temp3,axis=1)
        Wg = []
        Bg = []
        for i in reversed(range(len(self.Layers))):
            temp1, temp2, temp3 = self.Layers[i].back_propogate(temp3)
            Wg.insert(0,temp1)
            Bg.insert(0,temp2)
        return Wg,Bg
            
    def _update_weights(self):
        for i in range(len(self.Layers)):
            self.Weights[i] = self.Weights[i] - self.learning_rate * self.WGrads[i]
            self.Bias[i] = self.Bias[i] - self.learning_rate * self.BGrads[i]
            
    def predict(self,p):
        temp = p
        for i in range(len(self.Layers)):
            if i==0:
                temp = self.Layers[i].forward_propogate(temp,self.Weights[i],self.Bias[i],'relu')
            elif i==self.n_layers:
                temp = self.Layers[i].forward_propogate(temp,self.Weights[i],self.Bias[i],choice='sigm')
            else:
                temp = self.Layers[i].forward_propogate(temp,self.Weights[i],self.Bias[i],'relu')
        return temp

In [15]:
np.random.seed(1)
net = Neural_Net([3,4])
inputs = [[-4,-2],[2,3],[1,1],[-6,-6],[4,5],[7,-10]]
outputs = [1,0,0,1,1,0]

net.train(inputs,outputs,epochs=300,learning_rate=0.3,printCost = True)
print(net.predict([-4,-2]))
print(net.predict([2,3]))
print(net.predict([1,1]))
print(net.predict([-6,-6]))
print(net.predict([4,5]))
print(net.predict([7,-10]))

[0.40769741424634703]
Epoch 1/300 Cost = 0.873116, Cost_Der = -1.545374
Epoch 51/300 Cost = 0.376985, Cost_Der = 0.080946
Epoch 101/300 Cost = 0.184066, Cost_Der = 0.007893
Epoch 151/300 Cost = 0.073285, Cost_Der = -0.038534
Epoch 201/300 Cost = 0.047335, Cost_Der = -0.027056
Epoch 251/300 Cost = 0.035321, Cost_Der = -0.022253
[0.9642939678717656]
[0.031334216146563884]
[0.004159636600556321]
[0.9642939678717656]
[0.9642939678717656]
[0.00019616054247881105]
